# 상황에 맞게 맞춤 설정
---

fit()은 굉장히 편리합니다만, 자신만의 학습 루프를 작성 해야할 때가 존재합니다.

GradientTape는 자동 미분을 계산해주는 방식으로, tensorflow2에선, Tape에 Gradient를 저장하는 방식으로 Backpropagation을 계산해줍니다.

이 챕터에서는 fit()의 기능을 계속 활용하면서, 맞춤 설정하는 방법을 배웁니다.


Keras의 핵슴 원칙은 복잡성을 점진적으로 공개하는것입니다.

높은 수준의 편의성을 유지하면서 작은 세부사항을 더 잘 제어할 수 있어야합니다.
(고수준 api로 안되는 부분부분만 저수준으로 접근 가능하도록 만들었다 정도?)

fit()을 사용자 정의 해야하는 경우 Model 클래스의 학습 단계 함수를 재정의 해야합니다.

그러면 평소처럼 fit()을 호출할 수 있으며 자체 학습 알고리즘을 실행합니다.

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import *

## Model subclassing
---

- keras.Model을 상속
- train_step(self, data) 메서드를 재 정의
- 딕셔너리 매핑 메트릭 이름을 현재 값으로 반환

train_step은 fit()와 유사한 업데이트를 구현합니다.

self.compiled_loss를 통해 loss를 계산합니다. 이는 compile()로 전달 된 loss함수를 래핑합니다.

self.compiled_metrics.update_state(y, y_pred)를 호출하여 compile()에서 전달된 메트릭의 상태를 업데이트 하고

self.metrics 결과를 쿼리하여 현재 값을 검색합니다.

In [2]:
class CustomModel(keras.Model):
    
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) #Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # Update Wegiths
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        #Update Metrics
        self.compiled_metrics.update_state(y, y_pred)
        
        return {m.name: m.result() for m in self.metrics}

In [3]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2298 - mae: 0.3862
Epoch 2/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2149 - mae: 0.3742
Epoch 3/3
32/32 [==============================] - 0s 1ms/step - loss: 0.2044 - mae: 0.3645
